In [74]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

#1

print('Question 1:')
df=pd.read_csv('seismic_bumps.csv')
#dropping 8 columns as most values are 0 and are giving singular cov matrix
df1=df.drop(df.loc[:,'nbumps':'nbumps89'].columns,axis = 1) 
df10,df11=df1[df1['class']==0],df1[df1['class']==1]
y0,y1=df10['class'],df11['class']  #x label
x0,x1=df10.drop(['class'],axis = 1),df11.drop(['class'],axis = 1)
x_train0,x_test0,y_train0,y_test0=train_test_split(x0,y0,test_size=0.3,random_state=42, shuffle=True)
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.3,random_state=42, shuffle=True)
x_train,x_test = pd.concat((x_train0, x_train1), axis = 0),pd.concat((x_test0, x_test1), axis = 0)
y_train,y_test = pd.concat((y_train0, y_train1), axis = 0),pd.concat((y_test0, y_test1), axis = 0)
train,test = pd.concat((x_train, y_train), axis = 1),pd.concat((x_test, y_test), axis = 1)
train.to_csv('seismic-bumps-train.csv')
test.to_csv('seismic-bumps-test.csv')

# a & b

print('a & b:')
BestAcc = []
def knnc( x_train, y_train, x_test, y_test): #defining knnc function to see accuracies of different k values
    bestA = 0
    for k in [1,3,5]:
        knn1 = KNeighborsClassifier(n_neighbors=k)
        knn1.fit(x_train, y_train)
        y_pred = knn1.predict(x_test)
        print('Confusion matrix for k =', k)
        print(confusion_matrix(y_test, y_pred))
        a = accuracy_score(y_test, y_pred)
        print("Accuracy Score is",a,'\n')
        if a > bestA:
            bestA = round(a,4)
            bestk = k
    BestAcc.append(bestA)
    print('maximum accuracy for k= ', bestk,'at ',bestA, '\n')

knnc( x_train, y_train, x_test, y_test) #function call


#2

print('Question 2:')
#Min_Max Normalization exluding class attribute
x_trainnormal=((x_train-x_train.min())/(x_train.max()-x_train.min()))*(1-0)+0 
train_normal = pd.concat((x_trainnormal,y_train),axis = 1) #included class attribute
#Min_Max Normalization of test data using min, max of train dataset
x_testnormal=((x_test-x_train.min())/(x_train.max()-x_train.min()))*(1-0)+0
test_normal = pd.concat((x_testnormal,y_test),axis = 1)
train_normal.to_csv('seismic-bumps-train-Normalised.csv')
test_normal.to_csv(' seismic-bumps-test-normalised.csv')

# a & b

knnc(x_trainnormal, y_train,x_testnormal,y_test)

#3
print('Question 3:')

X_train=pd.read_csv('seismic-bumps-train.csv')
X_test=pd.read_csv('seismic-bumps-test.csv')

C0=X_train[X_train['class']==0][X_train.columns[0:-1]]
Mean_C0=C0.mean().values;Cov_C0=C0.cov().values

C1=X_train[X_train['class']==1][X_train.columns[0:-1]]
Mean_C1=C1.mean().values;Cov_C1=C1.cov().values
P_C0=len(C0)/(len(C0)+len(C1))
P_C1=len(C1)/(len(C0)+len(C1))
d=len(X_test.columns)-1

Predicted_class=[]
for x in X_test[X_test.columns[0:-1]].values:
    
    p_x_C0=1/(((2*np.pi)**(d/2))*np.linalg.det(Cov_C0)**0.5)*np.e**(-0.5*np.dot(np.dot((x-Mean_C0).T,np.linalg.inv(Cov_C0)),(x-Mean_C0)))
    p_x_C1=1/(((2*np.pi)**(d/2))*np.linalg.det(Cov_C1)**0.5)*np.e**(-0.5*np.dot(np.dot((x-Mean_C1).T,np.linalg.inv(Cov_C1)),(x-Mean_C1)))
    P_x=p_x_C0*P_C0+p_x_C1*P_C1
    
    P_C0_x=p_x_C0*P_C0/P_x
    P_C1_x=p_x_C1*P_C1/P_x
    
    if P_C0_x>P_C1_x:Predicted_class.append(0)
    else:Predicted_class.append(1)
    
print('Confusion Matrix :')
print(confusion_matrix(x_test[x_test.columns[-1]],Predicted_class),'\n')
print('Accuracy score :')
print(round(accuracy_score(x_test[x_test.columns[-1]],Predicted_class),4),'\n')
BestAcc.append(round(accuracy_score(x_test[x_test.columns[-1]],Predicted_class),4))
#4
print('Question 4:')    
data = { 'Classifier': ['K-NN Classifier','K-NN Classifier(normalized data)','Bayes Classifier'],
        'Accuracy Score':BestAcc}
table = pd.DataFrame(data,columns =['Classifier','Accuracy Score'],index=[1,2,3])
print(table)


Question 1:
a & b:
Confusion matrix for k = 1
[[671  54]
 [ 46   5]]
Accuracy Score is 0.8711340206185567 

Confusion matrix for k = 3
[[707  18]
 [ 47   4]]
Accuracy Score is 0.9162371134020618 

Confusion matrix for k = 5
[[718   7]
 [ 46   5]]
Accuracy Score is 0.9317010309278351 

maximum accuracy for k=  5 at  0.9317 

Question 2:
Confusion matrix for k = 1
[[678  47]
 [ 42   9]]
Accuracy Score is 0.8853092783505154 

Confusion matrix for k = 3
[[705  20]
 [ 44   7]]
Accuracy Score is 0.9175257731958762 

Confusion matrix for k = 5
[[718   7]
 [ 48   3]]
Accuracy Score is 0.9291237113402062 

maximum accuracy for k=  5 at  0.9291 

Question 3:
[1.33301303e+03 1.33510953e+00 1.40319716e+00 1.38898757e+00
 7.62098283e+04 4.90056838e+02 1.20822972e+01 3.54233274e+00
 1.10716400e+00 4.94174067e+03 4.37460036e+03]
Confusion Matrix :
[[408  12   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 